# Lab 6 

Ali Tejani, amt3639

Caroline Yao, chy253

Allen Hwang, ah45755

## Problem 1

### Estimating beta with linear regression

In [83]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge

In [84]:
n = 51
p = 50

In [154]:
def estimate_b(reg,n,p,seed=None):
    np.random.seed(seed=seed)
    b_hat = []
    for _ in range(100):
        X = np.random.normal(0,1,(n,p))
        e = np.random.normal(0,.25,(n,1))
        b = np.ones((p,1))
        y = np.dot(X,b) + e
        reg.fit(X,y)
        reg.coef_
        b_hat.append(reg.coef_)
    b_hat = np.array(b_hat)
    print b_hat.mean(axis=0).mean()
    print b_hat.var(axis=0).mean()

In [165]:
estimate_b(LinearRegression(fit_intercept=False),n,p,seed=1)

1.00655839826
0.227150750588


In [166]:
alphas = [0.01,0.1,1,10,100]
for a in alphas:
    estimate_b(Ridge(alpha=a),n,p,seed=1)

0.982576226953
0.0641155927754
0.950031875305
0.0392228415033
0.863646895296
0.0623827708922
0.642854600564
0.091449034224
0.268810522308
0.0409478695197


## Problem 2: Chapter 6, Problem 9

### a.  Split the data set into a training set and a test set

In [219]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [220]:
college = pd.read_csv('College.csv',index_col=0)
college['Private'] = college['Private'].replace('Yes',1).replace('No',0)
collegeX = college[[i for i in college.axes[1] if i != 'Apps']]
collegeY = college['Apps']
college_trainX,college_testX,college_trainY,college_testY = train_test_split(collegeX,collegeY,test_size=0.5)

### b. Fit a linear model using least squares on the training set, and report the test error obtained

In [241]:
def reg_score(reg):
    reg.fit(college_trainX,college_trainY)
    print reg.score(college_testX,college_testY)

In [262]:
reg_score(LinearRegression())

0.886856495002


### c.  Fit a ridge regression model on the training set, with λ chosen by cross-validation. Report the test error obtained.

In [278]:
from sklearn.linear_model import RidgeCV,LassoCV
ridge_reg = RidgeCV(alphas=[0.001,0.005,0.01,0.05,0.1,0.5])
reg_score(ridge_reg)
ridge_reg.alpha_

0.886847105855


0.5

### d.  Fit a lasso model on the training set, with λ chosen by crossvalidation. Report the test error obtained, along with the number of non-zero coefficient estimates.

In [281]:
lasso_reg = LassoCV(alphas=[0.001,0.005,0.01,0.05,0.1,0.5,1,10])
reg_score(lasso_reg)
lasso_reg.coef_

0.886492666461


array([ -3.03098103e+02,   1.72945804e+00,  -1.64257185e+00,
         3.94289481e+01,  -9.48374397e+00,   1.35788762e-01,
         1.29847836e-02,  -7.90511462e-02,   1.28427761e-01,
         2.28751163e-02,   1.58910211e-01,  -9.61633493e+00,
        -0.00000000e+00,   2.69949313e+01,   8.76543730e-01,
         6.50859085e-02,   5.33066548e+00])